<a href="https://colab.research.google.com/github/sanjaynagi/AnoExpress/blob/main/workflow/notebooks/differential-expression-meta-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

install.packages(c("pheatmap", "data.table", "ggrepel", "openxlsx", "tidyverse", "plotly", "RColorBrewer"))

In [9]:
library(DESeq2)
library(pheatmap)
library(data.table)
library(ggrepel)
library(openxlsx)
library(glue)
library(RColorBrewer)
library(tidyverse)
library(plotly)

## AnoExpress - Differential expression meta-analysis with DESeq2
In this notebook, we perform the differential expression analysis for AnoExpress.

In [10]:
dir.create("results/genediff", recursive=TRUE)

Warning message in dir.create("results/genediff", recursive = TRUE):
“'results/genediff' already exists”


In [11]:
round_df = function(df, digits) {
  #' This function rounds all the numeric columns of a data.frame
  nums = vapply(df, is.numeric, FUN.VALUE = logical(1))
  df[,nums] = round(df[,nums], digits = digits)
  (df)
}

diff_exp = function(analysis, names_df){
  # this function runs the diff exp analysis for one AnoExpress analysis

  results_list = list()
  nsig_list = list()

  # load metadata
  metadata = fread("https://raw.githubusercontent.com/sanjaynagi/AnoExpress/main/config/sample_metadata.tsv") %>% as.data.frame()
  metadata[(metadata$batch == 1) &(metadata$species == 'arabiensis'), 'batch'] = 0
  metadata$batch = factor(metadata$batch)

  # load analysis
  counts = fread(f("https://github.com/sanjaynagi/AnoExpress/raw/main/results/log2counts.{analysis}.tsv"), sep="\t") %>%
    as.data.frame() %>%
    column_to_rownames("GeneID") %>%
    2^. %>%
    mutate_if(is.numeric, as.integer)

  # get boolean indexer for species depending on analysis
  if (analysis == 'gamb_colu'){
    sp_bool = metadata$species %in% c("gambiae", "coluzzii")
  } else if (analysis == 'gamb_colu_arab'){
    sp_bool = metadata$species %in% c("gambiae", "coluzzii", "arabiensis")
  } else if (analysis == 'gamb_colu_arab_fun'){
    sp_bool = metadata$species %in% c("gambiae", "coluzzii", "arabiensis", "funestus")
  } else if (analysis == 'fun'){
    sp_bool = metadata$species == "funestus"
  }

  print(analysis)
  # subset to analysis
  meta = metadata[sp_bool, ]
  print(dim(meta))
  print(dim(counts))

  # analyse each experiment separately
  for (experiment in unique(meta$batch)){
    if (experiment == 5){
      next
    }

    # quit if shape of arrays incorrect
    stopifnot(nrow(meta) == length(counts))

    # subset to batch
    meta2 = meta %>% filter(batch == experiment)
    counts2 = counts[, meta2$sampleID]

    # quit if order not correct
    stopifnot(all(meta2$sampleID == colnames(counts2)))

    # get res and sus for each comparison
    resistants = unique(meta2[meta2$resistance == 'resistant',]$condition)
    susceptibles = unique(meta2[meta2$resistance == 'susceptible',]$condition)
    comparisons = crossing(resistants, susceptibles)
    print(experiment)
    print(as.data.frame(comparisons))

    for (i in 1:nrow(comparisons)){
      res = comparisons[i, 'resistants']
      sus = comparisons[i, 'susceptibles']
      comp = glue("{res}_v_{sus}")
      print(comp)

      controls = which(meta2$condition %in% sus)
      cases = which(meta2$condition %in% res)

      idxs = c(controls, cases)
      subcounts = counts2[, idxs]
      subsamples = meta2[idxs,]

      # make treatment a factor with the 'susceptible' as reference
      subsamples$treatment = as.factor(subsamples$resistance)
      subsamples$treatment = relevel(subsamples$treatment, "susceptible")

      # make DESeq analysis
      print("subcounts shape")
      print(dim(subcounts))
      print(head(subcounts))
      print("subsamples shape")
      print(dim(subsamples))
      print(head(subsamples))
      dds = DESeqDataSetFromMatrix(countData = subcounts,
                                   colData = subsamples,
                                   design = ~ treatment)

      ###### estimate paramters and normalise
      dds = estimateSizeFactors(dds)
      dds = estimateDispersions(dds)
      dds = estimateDispersions(dds)
      cds = nbinomWaldTest(dds)
      results = results(cds, contrast = c("treatment", "susceptible", "resistant")) %>% as.data.frame()
      results = results[order(results$padj),] #order by pvalue
      results = results %>% mutate(log2FoldChange=log2FoldChange*-1)
      results = results %>% rownames_to_column("GeneID") %>% dplyr::mutate("FC" = (2^log2FoldChange))

      ### absolute difference
      #### Get rowsums of counts, grouping by case/control. Then get difference of counts and join with DE results
      readdiff = data.frame(t(rowsum(t(subcounts), group = subsamples$treatment, na.rm = T))) #transpose and get rowsums for each group
      readdiff$absolute_diff = readdiff[,"resistant"] - readdiff[,"susceptible"] #get difference
      readdiff = data.frame(readdiff) %>% rownames_to_column('GeneID')
      results = unique(left_join(results, readdiff[,c('GeneID','absolute_diff')]))

      # join DE results with normal gene names
      results = unique(left_join(results, names_df))
      results_list[[comp]] = results

      fwrite(results, glue("results/genediff/{comp}_diffexp.csv")) #write to csv
      #get number of sig genes
      res1 = results %>% filter(padj < 0.05) %>%
        count("direction" = FC > 1) %>%
        dplyr::mutate("direction" = case_when(direction == FALSE ~ "Downregulated, padj = 0.05",
                                              direction == TRUE ~ "Upregulated, padj = 0.05")) %>%
        dplyr::rename(!!glue("{comp}_ngenes") := "n")

      res2 = results %>% filter(padj < 0.001) %>%
        count("direction" = FC > 1) %>%
        dplyr::mutate("direction" = case_when(direction == FALSE ~ "Downregulated, padj = 0.001",
                                              direction == TRUE ~ "Upregulated, padj = 0.001")) %>%
        dplyr::rename(!!glue("{comp}_ngenes") := "n")

      nsig_list[[comp]] = bind_rows(res1, res2)
      cat("\n", glue("{comp} complete!"), "\n")
      }
  }
  #### write to excel file on diff sheets ####
  sheets = names(results_list)
  wb <- createWorkbook("Workbook")
  for (i in 1:length(sheets)){
    addWorksheet(wb, glue("{sheets[[i]]}"))
    writeData(wb, sheets[i], results_list[[i]], rowNames = FALSE, colNames = TRUE)
  }
  #### save workbook to disk once all worksheets and data have been added ####
  saveWorkbook(wb,file=f("results/genediff/{analysis}_genediff.xlsx"), overwrite = TRUE)
  # Join different comparisons together and write out number of sig genes
  purrr::reduce(nsig_list, inner_join) %>% fwrite(f("results/genediff/{analysis}_nsig_genes.tsv"), sep="\t", col.names = TRUE)

  fc_data = data.frame("GeneID" = results_list[[1]]$GeneID)
  pval_data = data.frame("GeneID" = results_list[[1]]$GeneID)
  for (i in 1:length(results_list)){
    name = sheets[i]
    name_var = glue("{name}_log2FoldChange")
    name_pval = glue("{name}_padj")
    df = results_list[[i]] %>%
      select(c("GeneID", "log2FoldChange")) %>%
      rename({{ name_var }} := log2FoldChange)

    pval_df = results_list[[i]] %>%
      select(c("GeneID", "padj")) %>%
      rename({{ name_var }} := padj)

    fc_data = fc_data %>% inner_join(df) %>% distinct()
    pval_data = pval_data %>% inner_join(pval_df) %>% distinct()
  }

  fc_data = fc_data %>% inner_join(names_df)
  pval_data = pval_data %>% inner_join(names_df)

  pval_data %>%
    select(-TranscriptID) %>%
    round_df(3) %>%
    distinct() %>%
    fwrite(., file=f("results/pvals.{analysis}.tsv"), sep="\t")

  fc_data %>%
    select(-TranscriptID) %>%
    round_df(2) %>%
    distinct() %>%
    fwrite(., file=f("results/fcs.{analysis}.tsv"), sep="\t")

  return(list(results_list, nsig_list))
}

#### **Run analyses**

In [12]:
f = glue
ag_pest_analyses = c("gamb_colu", "gamb_colu_arab", "gamb_colu_arab_fun")

AGAMnames_df = fread("https://github.com/sanjaynagi/rna-seq-pop/raw/master/resources/exampleGene2TranscriptMap.tsv", sep="\t") %>% distinct()
AFUNnames_df = fread("https://github.com/sanjaynagi/AnoExpress/raw/main/resources/AfunGenes2TranscriptMap.tsv", sep="\t") %>% distinct()

In [ ]:
# gambiae pest analyses
for (analysis in ag_pest_analyses){
  res_list = diff_exp(analysis, names_df = AGAMnames_df)
}

# funestus only
res = diff_exp(analysis = "fun", names_df = AFUNnames_df)

In [7]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] plotly_4.10.2               lubridate_1.9.2            
 [3] forcats_1.0.0               stringr_1.5.0              
 [5] dplyr_1.1.2                 p